# Getting Started: Basic Tools for the DESI Survey QA Dashboard

This notebook describes some of the basic concepts and tools that we'll use to create a DESI survey QA dashboard.

# Bokeh Plotting Library

Bokeh is a python plotting library focused on making interactive plots to view in a web browser.
Compared to matplotlib/pylab, bokeh is much better at creating interactive
plots (which we will use for the dashboard), albeit a little more complicated.
We'll initially develop our plots in jupyter notebooks and then later port that
to python library code that can export them to HTML pages.

In [1]:
#- Basic imports
import numpy as np
import bokeh.plotting as bk
bk.output_notebook()

Loading BokehJS ...

In [2]:
x = np.random.normal(size=500)
y = np.random.normal(size=500)

fig1 = bk.figure(width=300, height=300)
fig1.scatter(x, y)

x = np.linspace(0, 4*np.pi, 30)
y = np.sin(x)
fig2 = bk.figure(width=500, height=300)
fig2.line(x, y)
fig2.circle(x, y, color='orange', size=10)

bk.show(bk.Row(fig1, fig2))

## FITS files and astropy Tables

FITS (Flexible Image Transport System) is the most common data format in astronomy, in use for almost 4 decades.  FITS files contain multiple "Header Data Units" (HDUs), where each HDU contains an ASCII "header" with keyword/value pairs of metadata followed by binary data that can be either an N-dimensional array or a table with named columns and rows of data.  For this project we'll mainly use the binary tables, and the `astropy.table.Table` interface to them.

In [3]:
from astropy.table import Table
tiles = Table.read('desi-tiles.fits')
tiles[0:5]

TILEID,RA,DEC,PASS,IN_DESI,EBV_MED,AIRMASS,STAR_DENSITY,EXPOSEFAC,PROGRAM,OBSCONDITIONS
int32,float64,float64,int16,int16,float32,float32,float32,float32,bytes6,int32
1,304.11,16.57,0,0,0.18588032,1.067882,25493.762,3.3637536,DARK,1
2,306.47,15.11,0,0,0.1337724,1.0762441,17306.73,2.4738233,DARK,1
3,308.78999999999996,13.63,0,0,0.08875092,1.0855807,13923.197,1.9014858,DARK,1
4,335.03,19.88,0,1,0.051561452,1.0518898,3973.278,1.4578364,DARK,1
5,313.31999999999994,10.62,0,0,0.08852036,1.107391,8475.746,1.9466255,DARK,1


In [4]:
#- Trim tiles to just the ones that are observable by DESI
keep = tiles['IN_DESI'] > 0
print('Keeping {} of {} tiles'.format(np.count_nonzero(keep), len(tiles)))
tiles = tiles[keep]

Keeping 16071 of 30898 tiles


In [5]:
#- Read exposures
exposures = Table.read('exposures.fits')

#- Remove calibration exposures
keep = exposures['PROGRAM'] != 'CALIB'
exposures = exposures[keep]

#- Show the first 5 exposures
exposures[0:5]

EXPID,MJD,EXPTIME,TILEID,SNR2FRAC,AIRMASS,SEEING,TRANSP,SKY,PROGRAM,NIGHT,FLAVOR,MOONFRAC,MOONALT,MOONSEP,RA,DEC,PASS
int32,float64,float32,int32,float32,float32,float32,float32,float32,bytes6,bytes8,bytes7,float64,float64,float64,float64,float64,int16
6,58821.063888888886,1040.0,24346,0.5039122,1.000185,1.0285894,0.9708182,1.0,GRAY,20191203,science,0.5026966578889441,45.11630841034253,45.65544084275721,344.53999999999996,32.61,4
7,58821.07662037016,690.0,24346,0.8773183,1.000185,1.2763733,0.80424196,1.0,GRAY,20191203,science,0.503887999439621,44.84625076956015,45.59938026687308,344.53999999999996,32.61,4
8,58821.08599537001,710.0,24084,0.34194383,1.0013524,1.1137799,0.8847155,1.0,GRAY,20191203,science,0.5047652600370685,44.38640235831237,46.05044348649063,348.0899999999999,32.84,4
9,58821.09490740691,710.0,24084,0.67199266,1.0013524,1.060869,0.92004615,1.0,GRAY,20191203,science,0.5055991991229509,43.74809777799925,46.006151207491115,348.0899999999999,32.84,4
10,58821.1038194438,470.0,24084,0.77594596,1.0013524,1.2519054,0.6598147,1.0,GRAY,20191203,science,0.5064331382088333,42.924178482303326,45.961337452300675,348.0899999999999,32.84,4


## Astronomical coordinates and terms

Right Ascension (**RA**) is a longitude-like coordinate on the sky, while declination (**dec**) is a latitude-like coordinate.

The Modified Julian Date (**MJD**) is the number of days since Nov 17, 1858, and is often used to record the start time of an observation as a floating point number.

**Airmass** is the amount of air through which the telescope was looking, normalized to airmass=1 when looking straight up.  It is approximately 1/cos(theta) where theta=the angle measured from straight up (the "zenith").  e.g. looking at 30 degrees above the horizon = 60 degrees from the zenith means airmass = 1/cos(60) = 2, i.e. looking through twice as much atmosphere as looking straight up.  Generally we want to observe through less air.

**Seeing** = amount of blur due to atmospheric turbulance and telescope optics.  It is measured in units of arcsec = 1/3600 degrees.  Seeing less than 1 arcsec is generally pretty good; seeing greater than 1 arcsec is a somewhat disappointing night.  For comparison, a human with good eyesight has effective seeing of 1 arcminute = 60 arcsec.  i.e. a good telescope can see ~60x sharper than a human.

## Example: plotting observed tiles

In [ ]:
#- plot all tiles
fig = bk.figure(width=600, height=300)
fig.circle(tiles['RA'], tiles['DEC'], color='gray', size=1)

#- np.in1d is a very handy function for determining which items in one array are present in another
#- use that to determine which tiles have already been observed
observed = np.in1d(tiles['TILEID'], exposures['TILEID'])
fig.circle(tiles['RA'][observed], tiles['DEC'][observed], color='blue', size=3)

bk.show(fig)

# Exercises

* Read the Bokeh [quickstart guide](https://bokeh.pydata.org/en/latest/docs/user_guide/quickstart.html#).
* Work through the [Bokeh tutorials](https://mybinder.org/v2/gh/bokeh/bokeh-notebooks/master?filepath=tutorial%2F00%20-%20Introduction%20and%20Setup.ipynb) 1-6 and 10.  7-9 and 11 are optional.
* Implement at least one of the following even if you haven't finished all the tutorials:
  1. Write a function that plots the number of tiles observed vs. time.
  2. Make a plot of the sky showing which tiles are observed like above, but additionally highlight the tiles that were observed on a specified night, e.g. on `int(MJD) = 58838`.  Color code the progression of the exposures of otherwise somehow indicate their order.
    * Bonus: add an interactive slider to move through the nights and update the plot on the fly
  3. Make a plot of the sky showing which tiles are observed like above, but add interactivity so that hovering over a tile shows what exposures observed that tile.
* Think about: the above plot shows the observed tiles (blue) overlaid on top of the locations all tiles (gray).  When zoomed out, this gives the impression that we are done with a large fraction of sky, but if you zoom in you will see that there are many unobserved gray tile locations that are obscured by the blue points.  How could you represent the data to give a better sense of actual completion for each part of the sky?